In [ ]:
import json
from pathlib import Path
import os
import re

In [ ]:
with open("/workdir/mlir-seeds/command_mapping.json", "r") as f:
    cmd_string_mapping = json.load(f)

In [ ]:
def get_cmd(text):
    # for now let's only consider mlir-opt
    pattern = r"^(mlir-opt) (.*?)[|\n]"
    matches = re.findall(pattern, text)
    if len(matches) < 1:
        return None
    if len(matches) > 1:
        #print(text)
        #raise ValueError("Expected at most one match")
        return None
    command, args = matches[0]
    return command, args.split()

cmd_mapping = dict()
for filename, command_strings in cmd_string_mapping.items():
    # concatenate command strings
    full_string = ""
    for cmd_str in command_strings:
        matches = re.findall(r"//\s+RUN:\s+(\S.*)", cmd_str)
        if len(matches) < 1:
            break
        if len(matches) > 1:
            raise ValueError("Expected at most one match")
        full_string += matches[0]
    full_string = full_string.replace("\\", "")
    #if "mlir-opt" in full_string:
    #    print(full_string)
    #    print("----")
    try:
        cmd_mapping[filename] = get_cmd(full_string)
    except:
        print(filename)

# filter out non mlir-opt commands
cmd_mapping = {file: cmdargs for file, cmdargs in cmd_mapping.items() if cmdargs is not None}
cmd_mapping

In [ ]:
dialects = [
   "acc", "affine", "amdgpu", "amx", "arith", "arm_neon", "arm_sme", "arm_sve", "async", "bufferization", "builtin", "cf", "complex", "dlti", "emitc", "func", "gpu", "index", "irdl", "linalg", "llvm", "math", "memref", "mesh", "ml_program", "nvgpu", "nvvm", "omp", "pdl", "pdl_interp", "quant", "rocdl", "scf", "shape", "sparse_tensor", "spirv", "tensor", "test_dyn", "tosa", "transform", "ub", "vector", "x86vector"
]

In [ ]:
dialect_associations = dict()
for cmd, args in cmd_mapping.values():
    for arg in args:
        possible_match = re.match(r"-convert-([a-z]+)-to", arg)
        if possible_match:
            dialect = possible_match.group(1)
            if dialect not in dialect_associations:
                dialect_associations[dialect] = []
            if arg not in dialect_associations[dialect]:
                dialect_associations[dialect].append(arg)
        possible_match = re.match(r"-([a-z]+)", arg)
        if possible_match:
            dialect = possible_match.group(1)
            if dialect in dialects:
                if dialect not in dialect_associations:
                    dialect_associations[dialect] = []
                if arg not in dialect_associations[dialect]:
                    dialect_associations[dialect].append(arg)
with open("/workdir/mlir-dialect-associations.json", "w") as f:
    json.dump(dialect_associations, f)